In [ ]:
import sys
import os
# Append the library path to PYTHONPATH, so library can be imported.
sys.path.append(os.path.dirname(os.getcwd()))
import shutil

import numpy as np
import pandas as pd

from library import network as nw
from library import common as cm

In [ ]:
%run setup.py

np.random.seed(RANDOM_SEED)

In [ ]:
%run Load_Clean_aux.py
max_period = max([int(s[6:]) for s in df.columns if 'period' in s])

#### Choose features and hypers

In [ ]:
hypers = {
    'nodes_per_layer': (30, 30),
    'reg_alpha': 1e-3,
    'lr': 1e-4,
    'epochs': 3,
    'outact': OUTACT
}

In [ ]:
if FEATURE_SET == 'normal_feature':
    ori_fea = ['M0', 'tau0_implvol0']
    sub_res = res_dir + 'Network/Normal_Feature/'

    if OUTACT == 'normcdf':
        sub_res  = sub_res[:-1] + '_CDF/'

if FEATURE_SET == 'delta_vega':
    ori_fea = ['delta_bs', '1_over_sqrt_tau', 'vega_n']
    sub_res = res_dir + f"Network/Delta_Vega/"
    
    if OUTACT == 'normcdf':
        sub_res  = sub_res[:-1] + '_CDF/'
        
        
if FEATURE_SET == 'delta_vega_vanna':
    ori_fea = ['delta_bs', '1_over_sqrt_tau', 'vega_n', 'vanna_n']
    sub_res = res_dir + 'Network/Delta_Vega_Vanna/'

    
if FEATURE_SET == 'spot_strike':
    ori_fea = ['S0_n', 'K_n', 'tau0_implvol0']
    sub_res = res_dir + 'Network/Spot_Strike/'

if FEATURE_SET == 'spot_strike_2':
    ori_fea = ['S0', 'K', 'tau0_implvol0']
    sub_res = res_dir + 'Network/Spot_Strike_2/'
    

if VIX:
    ori_fea += ['fake_vix']

use_fea = [x + '_t' for x in ori_fea] + ['cp_int']

if AGG_SIDE_FLAG:
    use_fea += ['AggressorSide']

os.makedirs(sub_res, exist_ok=True)
# Copy data setup file from the clean data folder, and then append network setup.
shutil.copy('setup.py', sub_res)
print(f'Result is stored at:\n{sub_res}')

#### Step 4: Train a network with given hypers

In [ ]:
if not PERMUTE:
    """
    Here, we use a bit different structure as the simulation data.
    In case of no permutation, there are only one ckp, history, plot folder.
    In case of permuation, in each of ckp, history and plot folders, we have many permutation folders.

    As for the pnl folder:
    In case of no permutation, only one pnl file.
    in case of permutations, many pnl files.
    This is consistent as the regression folder.
    """
    sub_res_dir = {
        'ckp': sub_res + 'ckp/',
        'history': sub_res + 'history/',
        'pnl': sub_res + 'pnl/',
        'plot': sub_res + 'plot/'}
    
    for key, value in sub_res_dir.items():
        os.makedirs(value, exist_ok=True)
        
    df = nw.rolling_net(
        df,
        ori_fea=ori_fea,
        use_fea=use_fea,
        end_period=max_period,
        hypers=hypers,
        sub_res_dir=sub_res_dir)
    
    cm.store_pnl(df, df['delta_nn'], sub_res_dir['pnl'] + 'pnl.csv')

In [ ]:
df.loc[3196783]

In [ ]:
if PERMUTE:
    for p in range(NUM_PERMUTE):
        """
        We keep all the pnl in the same folder, because each
        permute only has one pnl files and we can use the same code 
        for evaluation. But ckp, history, and plots are in 
        different permutations sub folders respectively.
        """
        sub_res_dir = {
            'ckp': sub_res + f'ckp/permute{p}/',
            'history': sub_res + f'history/permute{p}/',
            'pnl': sub_res + f'pnl/',
            'plot': sub_res + f'plot/permute{p}/'
        }
        
        for key, value in sub_res_dir.items():
            os.makedirs(value, exist_ok=True)
        
        df_permute = df.copy()
        df_permute = cm.rolling_permute(df_permute, random_seed=i+100)

        df_permute = nw.rolling_permute(
            df_permute, 
            ori_fea=ori_fea, 
            use_fea=use_fea, 
            hypers=hypers, 
            sub_res_dir=sub_res_dir)
        
        cm.store_pnl(df_permute, df_permute['delta_nn'], sub_res_dir['pnl'] + f'pnl{p}.csv')

In [ ]:
with open(f'{sub_res}additional_paras.txt', 'a') as file:
    file.write('\n\nThe following is network setup.\n')
    file.write(f'Date and time = {datetime.datetime.now()}\n')
    for n, x in [
        ('Learning rate', hypers['lr']),
        ('L2 regularization alpha', hypers['reg_alpha']),
        ('Output node activation', hypers['outact']),
        ('Nodes per layer', hypers['nodes_per_layer']),
        ('Number of training epochs', hypers['epochs'])
    ]:
        file.write(f'{n} = {x}\n')